In [1]:
!pip install huggingface_hub transformers datasets torch accelerate trl hf_transfer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 25.9 MB/s eta 0:00:00--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 105.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 126.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from huggingface_hub import login, snapshot_download
import os
import sys

def download_model(repo_id):
    # Set environment variable for fast transfer
    os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
    
    # Login (this will prompt for your token or you can pass it directly)
    login(token=os.getenv('HF_API_KEY'))
    
    # Download the entire repository
    repo_path = snapshot_download(repo_id=repo_id)
    
    print(f"Repository downloaded to: {repo_path}")


In [4]:
download_model("thehunter911/test")

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.78G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.33k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

Repository downloaded to: /root/.cache/huggingface/hub/models--thehunter911--test/snapshots/af5461f343ceea7be322270d2ad65f65fb08ba42


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "thehunter911/test"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "How many r in raspberry"
messages = [
    {"role": "system", "content": "You are a helpful and harmless assistant. You are Qwen developed by Alibaba. You should think step-by-step."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=5012
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
print(response)

think
Okay, so I need to figure out how many 'r' sounds are in the word "raspberry." Let me try to break this down step by step.

First, I should probably write out the word and then count each 'r' sound. The word is "raspberry." Hmm, let's spell it again: r-a-s-p-b-e-r-r-y. Right. So, breaking it down letter by letter:

1. r (first letter)
2. a
3. s
4. p
5. b
6. e
7. r
8. r
9. y

So, counting the 'r's: there's an 'r' at the beginning, then another 'r' before the 'y.' That's two 'r's. Wait, but maybe I need to check if there are any other 'r's that I might have missed. Let me go through each letter again mentally:

Start with r (1)
Then a, s, p, b, e – no 'r's there.
Then r (2)
Then r (3)
Then y.

Yes, exactly three 'r's in the word "raspberry." So the answer is three.

Wait, but just to make sure, because sometimes words have silent letters or something. But in this case, all the 'r's are pronounced. So each 'r' is there for a reason. So three is correct.

Alternatively, perhaps I sho

In [10]:
another_prompt = "Every morning Aya goes for a $9$-kilometer-long walk and stops at a coffee shop afterwards. When she walks at a constant speed of $s$ kilometers per hour, the walk takes her 4 hours, including $t$ minutes spent in the coffee shop. When she walks $s+2$ kilometers per hour, the walk takes her 2 hours and 24 minutes, including $t$ minutes spent in the coffee shop. Suppose Aya walks at $s+\frac{1}{2}$ kilometers per hour. Find the number of minutes the walk takes her, including the $t$ minutes spent in the coffee shop."

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful and harmless assistant. You are Qwen developed by Alibaba. You should think step-by-step."},
    {"role": "user", "content": another_prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=5012
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(response)